In [ ]:
from copy import deepcopy
from graph import Graph
import random
import sys
import math
import matplotlib.pyplot as plt

# Karger

In [ ]:
def karger(graph):
    while len(graph) > 2:
        v = random.choice(list(graph.keys()))
        w = random.choice(graph[v])
        union(graph, v, w)
    return len(list(graph.values())[0])

def union(graph, v, w):
    for node in graph[w]:
        if node != v:
            graph[v].append(node)
        graph[node].remove(w)
        if node != v:
            graph[node].append(v)
    del graph[w]

def karger_iter(graph, iter):
    minCut = sys.maxsize;
    cut    = sys.maxsize;
    for _ in range(iter):
        G = deepcopy(graph)
        cut = karger(G)
        if cut < minCut:
            minCut = cut
    return minCut


# Ingênuo

In [ ]:
def naive(graph):
    min_cut = float('inf')
    for _ in range(len(graph)):
        i = random.randint(0, len(graph))
        cut = len(graph[i])
        if cut < min_cut:
            min_cut = cut
    return min_cut

def naive_iter(graph, iter):
    minCut = sys.maxsize;
    cut    = sys.maxsize;
    for _ in range(iter):
        G = deepcopy(graph)
        cut = naive(G)
        if cut < minCut:
            minCut = cut
    return minCut

# Rodando Kaarger `Nexec` vezes para obter o provavel corte mínimo

In [ ]:
def cutInNexec(G, Nexec):
    expectedCut = sys.maxsize
    auxCut = 0
    for _ in range(Nexec):
        auxCut = karger_iter(G, 10)
        if auxCut < expectedCut:
            expectedCut = auxCut
    return expectedCut

# `Niter` e teste empírico

Segundo os slides, uma generalização para o aumento da probabilidade de corte mínimo é $T = (\frac{n}{2})\cdot\log{(\frac{1}{\alpha})}$, onde $n$ é a quantidade de vértices e $\alpha = (0,1)$. Então nosso `Niter` será $T$ e o $\alpha$ terá um ajuste manual para cada grafo.

Tabela de $\alpha$

| Grafo         | $\alpha$|
|   ---         | --- |
| graph_type1_2 | 0.3 |
| graph_type1_3 | 0.4 |
| graph_type2_1 | 0.3 |
| graph_type2_2 | 0.2 |

In [ ]:
def calcProb(G, alpha):
    Nexec       = 10_000
    Niter       = math.ceil((len(G)/2)*math.log(1/alpha))
    expectedCut = cutInNexec(G, Nexec)

    probKarger  = [0.0 for _ in range(Niter)]
    probNaive   = [0.0 for _ in range(Niter)]

    for iter in range(Niter):
        for _ in range(Nexec):
            cut = karger_iter(G, iter)
            if cut == expectedCut:
                probKarger[iter] += 1
            cut = naive_iter(G, iter)
        probKarger[iter] /= Nexec

    for iter in range(Niter):
        for _ in range(Nexec):
            cut = naive_iter(G, iter)
            if cut < expectedCut:
                probNaive[iter] += 1
        probNaive[iter] /= Nexec
    return probKarger, probNaive


# Plotando Graficos

In [ ]:
def plotarProb(dataKarger, dataIngenuo, titulo):
    plt.plot(dataKarger, color="blue", label="Karger")
    plt.plot(dataIngenuo, color="orange", label="Ingênuo")
    plt.legend()
    plt.title(f"probabilidade empírica do grafo \"{titulo}\"")
    plt.xlabel( "N iter" )
    plt.ylabel( "Sucesso da Probabilidade empírica")
    plt.grid()
    plt.show()

In [ ]:
probKarger, probNaive = calcProb(Graph('in/p3/graph_type1_1').get_adj_list_as_dict(), 0.6)
plotarProb(probKarger, probNaive, 'graph_type1_1')

In [ ]:
probKarger, probNaive = calcProb(Graph('in/p3/graph_type1_2').get_adj_list_as_dict(), 0.4)
plotarProb(probKarger, probNaive, 'graph_type1_2')

In [ ]:
probKarger, probNaive = calcProb(Graph('in/p3/graph_type1_3').get_adj_list_as_dict(), 0.9)
plotarProb(probKarger, probNaive, 'graph_type1_3')

In [ ]:
probKarger, probNaive = calcProb(Graph('in/p3/graph_type2_1').get_adj_list_as_dict(), 0.3)
plotarProb(probKarger, probNaive, 'graph_type2_1')

In [ ]:
probKarger, probNaive = calcProb(Graph('in/p3/graph_type2_2').get_adj_list_as_dict(), 0.3)
plotarProb(probKarger, probNaive, 'graph_type2_2')

In [ ]:
probKarger, probNaive = calcProb(Graph('in/p3/graph_type2_3').get_adj_list_as_dict(), 0.3)
plotarProb(probKarger, probNaive, 'graph_type2_3')